<a href="https://colab.research.google.com/github/Aung505/myproject/blob/master/%D0%A0%D0%9A_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

РК №2 

**Аунг Каунг Кхант, ИУ5И-21М**

Indented block

Задача №1. Классификация текстов на основе методов наивного Байеса.
Задание:
Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета. Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.

Необходимо сформировать признаки на основе CountVectorizer или TfidfVectorizer.

В качестве классификаторов необходимо использовать один из классификаторов, не относящихся к наивным Байесовским методам (например, LogisticRegression), а также Multinomial Naive Bayes (MNB), Complement Naive Bayes (CNB), Bernoulli Naive Bayes.

Для каждого метода необходимо оценить качество классификации с помощью хотя бы одной метрики качества классификации (например, Accuracy).

Сделате выводы о том, какой классификатор осуществляет более качественную классификацию на Вашем наборе данных.

In [0]:
import pandas as pd

df = pd.read_csv("/content/Reviews.csv", sep = ",")

In [2]:
df.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


In [3]:
del df['ProductId']
del df['UserId']
del df['HelpfulnessNumerator']
del df['HelpfulnessDenominator']
del df['Time']
del df['ProfileName']
del df['Id']
df.head(3)

,Score,Summary,Text
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...


In [4]:
df.dtypes

Score       int64
Summary    object
Text       object
dtype: object

In [5]:
# Проверка на пустые значения
df.isnull().sum()

Score       0
Summary    23
Text        0
dtype: int64

In [0]:
df = df.dropna(axis=0, how='any')

In [7]:
df.shape

(467525, 3)

In [0]:
# df3_ = df3.dropna(axis=0, how='any')

In [0]:
%matplotlib inline 

**Обработка данных**

In [0]:
from typing import Dict, Tuple
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np
import string

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['Text'], 
    df['Score'],
     test_size=0.4,
    random_state = 1
)

print("Training dataset: ", X_train.shape[0])
print("Test dataset: ", X_test.shape[0])

Training dataset:  280515
Test dataset:  187010


In [0]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res
def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [0]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [14]:
import warnings
warnings.filterwarnings('ignore')

sentiment(TfidfVectorizer(), LogisticRegression(C=5.0))

Метка 	 Accuracy
1 	 0.6861414606095457
2 	 0.28174160154267736
3 	 0.32069209039548024
4 	 0.2891837193613672
5 	 0.9351692243010301


In [15]:
sentiment(CountVectorizer(), MultinomialNB())

Метка 	 Accuracy
1 	 0.6397354801610121
2 	 0.20704354003856693
3 	 0.30388418079096047
4 	 0.3664642830372536
5 	 0.8727517342863149


In [16]:
sentiment(TfidfVectorizer(), MultinomialNB())

Метка 	 Accuracy
1 	 0.06578493387004025
2 	 0.0004059677255658175
3 	 0.00035310734463276836
4 	 0.005659245933588187
5 	 0.9993861677527853


In [18]:
sentiment(CountVectorizer(), ComplementNB())

Метка 	 Accuracy
1 	 0.7948821161587118
2 	 0.13356338171115395
3 	 0.2685734463276836
4 	 0.336294130874747
5 	 0.8638049190666386


In [19]:
sentiment(TfidfVectorizer(), ComplementNB())

Метка 	 Accuracy
1 	 0.7508913168487636
2 	 0.11803511620826145
3 	 0.1952683615819209
4 	 0.21209054793493742
5 	 0.9233466470464579


In [20]:
sentiment(CountVectorizer(binary=True), BernoulliNB())

Метка 	 Accuracy
1 	 0.48936170212765956
2 	 0.12554551913122908
3 	 0.25557909604519774
4 	 0.30945956075256725
5 	 0.827101114147572


In [21]:
sentiment(TfidfVectorizer(binary=True), BernoulliNB())

Метка 	 Accuracy
1 	 0.48936170212765956
2 	 0.12554551913122908
3 	 0.25557909604519774
4 	 0.30945956075256725
5 	 0.827101114147572


**Вывод:**

Методы классификации текстов, основанные на "наивном" Байесе работают не хуже чем логистическая регрессия. Логистическая регрессия - точность достигает даже 95 процентов для метки 5,70%-для 1, для остальных случаев результаты не очень хорошие. Во всех методах для метки 5 были достигнуты хорошие результаты - выше 82 процентов. Логистическая регрессия работает более плавно. Все методы в чем-то показывают лучше результат, а в чем-то хуже. Закономерности не наблюдается.